# Hyperparameter tuning by randomized-search

In the previous notebook, we showed how to use a grid-search approach to
search for the best hyperparameters maximizing the statistical performance
of a predictive model.

However, a grid-search approach has limitations. It does not scale when
the number of parameters to tune is increasing. Also, the grid will imposed
a regularity during the search which might be problematic.

In this notebook, we will present the another method to tune hyperparameters
called randomized search.

## Our predictive model

Let us reload the dataset as we did previously:

In [3]:
from sklearn import set_config

set_config(display="diagram")

In [5]:
import pandas as pd

adult_census = pd.read_csv("../datasets/adult-census.csv")

We extract the column containing the target.

In [7]:
target_name = "class"
target = adult_census[target_name]
target

0         <=50K
1         <=50K
2          >50K
3          >50K
4         <=50K
          ...  
48837     <=50K
48838      >50K
48839     <=50K
48840     <=50K
48841      >50K
Name: class, Length: 48842, dtype: object

We drop from our data the target and the `"education-num"` column which
duplicates the information with `"education"` columns.

In [9]:
data = adult_census.drop(columns=[target_name, "education-num"])
data.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States


Once the dataset is loaded, we split it into a training and testing sets.

In [11]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

We will create the same predictive pipeline as seen in the grid-search
section.

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer([
    ('cat-preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)

In [15]:
# for the moment this line is required to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42, max_leaf_nodes=4)),
])

model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat-preprocessor',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier',
                 HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                random_state=42))])

## Tuning using a randomized-search

With the `GridSearchCV` estimator, the parameters need to be specified
explicitly. We already mentioned that exploring a large number of values for
different parameters will be quickly untractable.

Instead, we can randomly generate the parameter candidates. Indeed,
such approach avoids the regularity of the grid. Hence, adding more
evaluations can increase the resolution in each direction. This is the
case in the frequent situation where the choice of some hyperparameters
is not very important, as for hyperparameter 2 in the figure below.

![Randomized vs grid search](../figures/grid_vs_random_search.svg)

Indeed, the number of evaluation points need to be divided across the
two different hyperparameters. With a grid, the danger is that the
region of good hyperparameters fall between the line of the grid: this
region is aligned with the grid given that hyperparameter 2 has a weak
influence. Rather, stochastic search will sample hyperparameter 1
independently from hyperparameter 2 and find the optimal region.

The `RandomizedSearchCV` class allows for such stochastic search. It is
used similarly to the `GridSearchCV` but the sampling distributions
need to be specified instead of the parameter values. For instance, we
will draw candidates using a log-uniform distribution because the parameters
we are interested in take positive values with a natural log scaling (.1 is
as close to 1 as 10 is).

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">Random search (with <tt class="docutils literal">RandomizedSearchCV</tt>) is typically beneficial compared
to grid search (with <tt class="docutils literal">GridSearchCV</tt>) to optimize 3 or more
hyperparameters.</p>
</div>

We will optimize 3 other parameters in addition to the ones we
optimized above:

* `max_iter`: it corresponds to the number of trees in the ensemble;
* `min_samples_leaf`: it corresponds to the minimum number of samples
  required in a leaf;
* `max_bins`: it corresponds to the maximum number of bins to construct the
  histograms.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last"><tt class="docutils literal">scipy.stats.loguniform</tt> can be used to generate floating numbers. To
generate random values for integer-valued parameters (e.g.
<tt class="docutils literal">min_samples_leaf</tt>) we can adapt is as follows:</p>
</div>

In [17]:
from scipy.stats import loguniform


class loguniform_int:
    """Integer valued version of the log-uniform distribution"""
    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)


Now, we can define the randomized search using the different distributions.
Executing 10 iterations of 5-fold cross-validation for random
parametrizations of this model on this dataset can take from 10 seconds to
several minutes, depending on the speed of the host computer and the number
of available processors.

In [19]:
%%time
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'classifier__l2_regularization': loguniform(1e-6, 1e3),
    'classifier__learning_rate': loguniform(0.001, 10),
    'classifier__max_leaf_nodes': loguniform_int(2, 256),
    'classifier__min_samples_leaf': loguniform_int(1, 100),
    'classifier__max_bins': loguniform_int(2, 255),
}

model_random_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=10,
    cv=5, verbose=1,
)
model_random_search.fit(data_train, target_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Wall time: 1min 33s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                sparse_threshold=0,
                                                                transformers=[('cat-preprocessor',
                                                                               OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                              unknown_value=-1),
                                                                               ['workclass',
                                                                                'education',
                                                                                'marital-status',
                                                                                'occupation',
                                                                                'relationship',
                                                                                'race',
                                                                                'sex',
                                                                                'native-country'])])),
                                             ('classifier',
                                              Hi...
                                        'classifier__learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E616E11610>,
                                        'classifier__max_bins': <__main__.loguniform_int object at 0x000001E617087130>,
                                        'classifier__max_leaf_nodes': <__main__.loguniform_int object at 0x000001E616135E50>,
                                        'classifier__min_samples_leaf': <__main__.loguniform_int object at 0x000001E616E11100>},
                   verbose=1)

Then, we can compute the accuracy score on the test set.

In [21]:
accuracy = model_random_search.score(data_test, target_test)

print(f"The test accuracy score of the best model is "
      f"{accuracy:.2f}")

The test accuracy score of the best model is 0.86


In [23]:
from pprint import pprint

print("The best parameters are:")
pprint(model_random_search.best_params_)

The best parameters are:
{'classifier__l2_regularization': 9.050211892574149,
 'classifier__learning_rate': 0.5989797175966926,
 'classifier__max_bins': 37,
 'classifier__max_leaf_nodes': 99,
 'classifier__min_samples_leaf': 11}



We can inspect the results using the attributes `cv_results` as we did
previously.

In [25]:
def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name

In [27]:
# get the parameter names
column_results = [
    f"param_{name}" for name in param_distributions.keys()]
column_results += [
    "mean_test_score", "std_test_score", "rank_test_score"]

cv_results = pd.DataFrame(model_random_search.cv_results_)
cv_results = cv_results[column_results].sort_values(
    "mean_test_score", ascending=False)
cv_results = cv_results.rename(shorten_param, axis=1)
cv_results

,l2_regularization,learning_rate,max_leaf_nodes,min_samples_leaf,max_bins,mean_test_score,std_test_score,rank_test_score
4,9.050212,0.59898,99,11,37,0.850973,0.002963,1
8,0.000004,0.043559,60,2,10,0.843357,0.003312,2
1,0.000217,0.073176,151,13,12,0.842156,0.003003,3
7,0.000032,0.13471,3,15,5,0.827878,0.001460,4
6,244.079461,0.050975,55,33,3,0.806366,0.001578,5
2,0.000353,1.312476,147,8,3,0.789768,0.004681,6
9,0.249574,0.001846,110,2,20,0.758947,0.000013,7
3,757.559308,8.39549,57,40,2,0.691871,0.069682,8
5,3.34625,4.712171,29,4,4,0.656930,0.093208,9
0,0.000022,2.798109,11,1,5,0.614860,0.173053,10


In practice, a randomized hyperparameter search is usually run with a large
number of iterations. In order to avoid the computation cost and still make a
decent analysis, we load the results obtained from a similar search with 200
iterations.

In [29]:
# model_random_search = RandomizedSearchCV(
#     model, param_distributions=param_distributions, n_iter=500,
#     n_jobs=2, cv=5)
# model_random_search.fit(df_train, target_train)
# cv_results =  pd.DataFrame(model_random_search.cv_results_)
# cv_results.to_csv("../figures/randomized_search_results.csv")

In [31]:
cv_results = pd.read_csv("../figures/randomized_search_results.csv",
                         index_col=0)

As we have more than 2 parameters in our grid-search, we cannot visualize the
results using a heatmap. However, we can us a parallel coordinates plot.

In [33]:
(cv_results[column_results].rename(
    shorten_param, axis=1).sort_values("mean_test_score"))

,l2_regularization,learning_rate,max_leaf_nodes,min_samples_leaf,max_bins,mean_test_score,std_test_score,rank_test_score
357,0.000026,3.075318,3,68,31,0.241053,0.000013,500
200,0.000444,6.236325,2,2,30,0.344629,0.207156,499
413,0.000001,8.828574,64,1,144,0.448205,0.253714,497
344,0.000003,7.091079,5,1,95,0.448205,0.253714,497
232,0.000097,9.976823,28,5,3,0.448205,0.253714,496
...,...,...,...,...,...,...,...,...
327,4.733808,0.036786,61,5,241,0.869673,0.002417,5
328,2.036232,0.224702,28,49,236,0.869837,0.000808,4
21,4.994918,0.077047,53,7,192,0.870793,0.001993,3
343,0.000404,0.244503,15,15,229,0.871339,0.002741,2


In [34]:
import numpy as np
import plotly.express as px

fig = px.parallel_coordinates(
    cv_results.rename(shorten_param, axis=1).apply({
        "learning_rate": np.log10,
        "max_leaf_nodes": np.log2,
        "max_bins": np.log2,
        "min_samples_leaf": np.log10,
        "l2_regularization": np.log10,
        "mean_test_score": lambda x: x}),
    color="mean_test_score",
    color_continuous_scale=px.colors.sequential.Viridis,
)
fig.show()


The parallel coordinates plot will display the values of the hyperparameters
on different columns while the performance metric is color coded. Thus, we
are able to quickly inspect if there is a range of hyperparameters which is
working or not.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">We <strong>transformed most axis values by taking a log10 or log2</strong> to
spread the active ranges and improve the readability of the plot.</p>
</div>

In particular for this hyper-parameter search, it is interesting to see that
the yellow lines (top performing models) all reach intermediate values for
the learning rate, that is, tick values between -2 and 0 which correspond to
learning rate values of 0.01 to 1.0 once we invert the log10 transform for
that axis.

It is possible to **select a range of results by clicking and holding on any
axis** of the parallel coordinate plot. You can then slide (move) the range
selection and cross two selections to see the intersections. You can undo a
selection by clicking once again on the same axis.

We also observe that it is not possible to select the highest performing
models by selecting lines of on the `max_bins` axis with tick values between
1 and 3.

The other hyper-parameters are not very sensitive. We can check that if we
select the `learning_rate` axis tick values between -1.5 and -0.5 and
`max_bins` tick values between 5 and 8, we always select top performing
models, whatever the values of the other hyper-parameters.


In this notebook, we have seen how randomized search offer a valuable
alternative to grid-search when the number of hyperparameters to tune is more
than two. It also alleviates the regularity imposed by the grid that might be
problematic sometimes.